In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load and preprocess the data

In [18]:
data_dir = "../data/images_train/"

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45, # these parameters can be tuned
    width_shift_range=0.2, # these parameters can be tuned
    height_shift_range=0.2, # these parameters can be tuned
    shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16, # batch size can be tuned
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation' 
)

Found 524 images belonging to 2 classes.


Found 92 images belonging to 2 classes.


# Load a pre-trained ResNet50 model

In [ ]:
# model = ResNet50(weights='imagenet')
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [19]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [20]:
# Unfreeze the last convolutional block
# ResNet50 typically has 'conv5_block3' as the last block
for layer in base_model.layers:
    if 'conv5_block3' in layer.name:
        layer.trainable = True

In [22]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x) # dropout can be tuned
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model

In [23]:
model.compile(optimizer=Adam(learning_rate=0.0001), # learning rate can be tuned
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model

In [28]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=4)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10, # epochs can be tuned
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    # callbacks=[early_stopping]
)

Epoch 1/10
32/32 [==============================] - 58s 2s/step - loss: 0.4860 - accuracy: 0.7854 - val_loss: 0.5106 - val_accuracy: 0.8000
Epoch 2/10
32/32 [==============================] - 60s 2s/step - loss: 0.4605 - accuracy: 0.7894 - val_loss: 0.8731 - val_accuracy: 0.5500
Epoch 3/10
32/32 [==============================] - 63s 2s/step - loss: 0.4520 - accuracy: 0.7835 - val_loss: 0.8282 - val_accuracy: 0.5750
Epoch 4/10
32/32 [==============================] - 63s 2s/step - loss: 0.4483 - accuracy: 0.7874 - val_loss: 1.2991 - val_accuracy: 0.4875
Epoch 5/10
32/32 [==============================] - 62s 2s/step - loss: 0.4428 - accuracy: 0.7933 - val_loss: 0.7887 - val_accuracy: 0.5750
Epoch 6/10
32/32 [==============================] - 60s 2s/step - loss: 0.4318 - accuracy: 0.8110 - val_loss: 0.7221 - val_accuracy: 0.6250
Epoch 7/10
32/32 [==============================] - 61s 2s/step - loss: 0.4729 - accuracy: 0.7776 - val_loss: 0.4018 - val_accuracy: 0.8625
Epoch 8/10
32/32 [==

Adam, learning rate 0.0001 val_accuracy: 0.6042

batch = 16, Adam, learning rate 0.001 accuracy 0.7321

In [29]:
test_dir = '../data/images_test'

predictions = []

for image_name in os.listdir(test_dir):
    image_id = image_name[:3]
    img_path = os.path.join(test_dir, image_name)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.

    # predict
    prediction = model.predict(img_array)
    predicted_class = int(prediction[0][0] <= 0.5) # 0 for kitchen, 1 for bathroom
    predictions.append([image_id,predicted_class])

predictions_df = pd.DataFrame(predictions, columns=['id', 'class'])
predictions_df.to_csv('image_prediction.csv', index=False)

1/1 [==============================] - 0s 98ms/step


In [30]:
predictions_df

,id,class
0,100,0
1,101,0
2,102,0
3,103,0
4,104,0
...,...,...
144,269,1
145,270,0
146,271,0
147,272,1


0, 0, 0, 0, 1, ..., 1, 0, 0, 1, 1

In [12]:
# train_generator.class_indices